In [ ]:
!pip install tensorflow-hub


  Obtaining dependency information for tensorflow-hub from https://files.pythonhosted.org/packages/6e/1a/fbae76f4057b9bcdf9468025d7a8ca952dec14bfafb9fc0b1e4244ce212f/tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/85.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/85.4 kB ? eta -:--:--
   ---------------------------- ----------- 61.4/85.4 kB 825.8 kB/s eta 0:00:01
   ---------------------------------------- 85.4/85.4 kB 963.8 kB/s eta 0:00:00


In [4]:
!pip install tensorflow-text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 34.9 MB/s eta 0:00:00


In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
df=pd.read_csv("/content/spam.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [8]:
spam_df=df[df['Category']=='spam']

In [9]:
ham_df=df[df['Category']=='ham']

In [10]:
ham_balanced=ham_df.sample(spam_df.shape[0])
ham_balanced.shape

(747, 2)

In [11]:
balanced_df=pd.concat([spam_df,ham_balanced])

In [12]:
balanced_df['Category'].value_counts()


spam    747
ham     747
Name: Category, dtype: int64

In [13]:
balanced_df['spam'] = balanced_df['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [14]:
balanced_df

,Category,Message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1
...,...,...,...
745,ham,Men like shorter ladies. Gaze up into his eyes.,0
5289,ham,Hey! Congrats 2u2. id luv 2 but ive had 2 go h...,0
1636,ham,You have come into my life and brought the sun...,0
3873,ham,I am joining today formally.Pls keep praying.w...,0


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test=train_test_split(balanced_df['Message'],balanced_df['spam'], stratify=balanced_df['spam'])

In [17]:
X_train

1486    (I should add that I don't really care and if ...
1449    As a registered optin subscriber ur draw 4 £10...
1163    Free entry in 2 a wkly comp to win FA Cup fina...
688     Dear,Me at cherthala.in case u r coming cochin...
1587    There are no other charges after transfer char...
                              ...                        
840     Last chance 2 claim ur £150 worth of discount ...
2059                     Ugh fuck it I'm resubbing to eve
3298    Todays Voda numbers ending 5226 are selected t...
4450    Urgent UR awarded a complimentary trip to Euro...
2800    I've told him that i've returned it. That shou...
Name: Message, Length: 1120, dtype: object

In [18]:
X_test

5501    PRIVATE! Your 2003 Account Statement for 07808...
1263    Hungry gay guys feeling hungry and up 4 it, no...
2181    Babe ! How goes that day ? What are you doing ...
5393    All done, all handed in. Don't know if mega sh...
3032    gonna let me know cos comes bak from holiday t...
                              ...                        
4258    important information 4 orange user . today is...
3666                Ha... U jus ate honey ar? So sweet...
4647    We tried to contact you re your reply to our o...
4673    Customer service announcement. We recently tri...
90      Yeah do! Don‘t stand to close tho- you‘ll catc...
Name: Message, Length: 374, dtype: object

In [19]:
y_train

1486    0
1449    1
1163    1
688     0
1587    0
       ..
840     1
2059    0
3298    1
4450    1
2800    0
Name: spam, Length: 1120, dtype: int64

In [20]:
y_test

5501    1
1263    1
2181    0
5393    0
3032    0
       ..
4258    1
3666    0
4647    1
4673    1
90      0
Name: spam, Length: 374, dtype: int64

In [21]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [25]:
text_input=tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)

layer = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
layer=tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model=tf.keras.Model(inputs=[text_input], outputs=[layer])

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [32]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=METRICS)

In [34]:
model.fit(X_train,y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 495s 14s/step - loss: 0.6430 - accuracy: 0.6268 - precision: 0.6224 - recall: 0.6446
Epoch 2/10
35/35 [==============================] - 485s 14s/step - loss: 0.5167 - accuracy: 0.7893 - precision: 0.7989 - recall: 0.7732
Epoch 3/10
35/35 [==============================] - 487s 14s/step - loss: 0.4528 - accuracy: 0.8420 - precision: 0.8285 - recall: 0.8625
Epoch 4/10
35/35 [==============================] - 487s 14s/step - loss: 0.3996 - accuracy: 0.8545 - precision: 0.8513 - recall: 0.8589
Epoch 5/10
35/35 [==============================] - 495s 14s/step - loss: 0.3683 - accuracy: 0.8884 - precision: 0.8705 - recall: 0.9125
Epoch 6/10
35/35 [==============================] - 505s 14s/step - loss: 0.3516 - accuracy: 0.8821 - precision: 0.8768 - recall: 0.8893
Epoch 7/10
35/35 [==============================] - 486s 14s/step - loss: 0.3358 - accuracy: 0.8920 - precision: 0.8791 - recall: 0.9089
Epoch 8/10
35/35 [=======================

In [36]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 159s 13s/step - loss: 0.2773 - accuracy: 0.9198 - precision: 0.9067 - recall: 0.9358


[0.27726316452026367,
 0.9197860956192017,
 0.9067357778549194,
 0.9358288645744324]

In [38]:
y_predicted=model.predict(X_test)
y_predicted=y_predicted.flatten()

12/12 [==============================] - 161s 13s/step


In [43]:
y_predicted=np.where(y_predicted >0.5, 1, 0)
y_predicted

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,

In [44]:
from sklearn.metrics import confusion_matrix, classification_report

cm=confusion_matrix(y_test, y_predicted )
cm

array([[169,  18],
       [ 12, 175]])

In [45]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92       187
           1       0.91      0.94      0.92       187

    accuracy                           0.92       374
   macro avg       0.92      0.92      0.92       374
weighted avg       0.92      0.92      0.92       374

